# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおけるトピックモデリングを目的としたもので、主要な問題はテキストデータから有意義なトピックを抽出し、それを視覚化することです。特に、ユーザーからのプロンプトとそれに対する応答を解析し、各プロンプトの潜在的なパターンを特定することで、チャットボットの応答を改善するための洞察を得ることを目指しています。

### 使用されている手法およびライブラリ

1. **テキスト処理とデータ前処理**:
   - **Pandas**: データフレームを操作し、トレーニングとテストデータを読み込み、空の応答を除外するために使用。
   - **literal_eval**: 文字列をPythonのリテラルに変換するために使用。

2. **トピックモデリング**:
   - **BERTopic**: トピックモデリングを行うためのライブラリで、以下の一連の処理を実施:
     - **SentenceTransformer**: 文を埋め込みベクトルに変換。
     - **UMAP**: 次元削減手法で、データの次元数を減少させるために使用。
     - **HDBSCAN**: 様々な形状のクラスタを見つけるための密度ベースのクラスタリング手法。
     - **CountVectorizer / TfidfVectorizer**: テキストデータを数値ベクトルに変換。
     - **c-TF-IDF**: 重み付けされたトピック表現を生成するために使われる。

3. **視覚化手法**:
   - トピックの重要度や関係性を示すために、バーチャートやヒートマップを生成。具体的には `visualize_barchart` や `visualize_heatmap` を使用。

4. **ガイド付きトピックモデリング**:
   - 修正されたタスクキーワードを用いて、ガイド付きトピックモデリングを実施し、特定のNLPタスクに関連するテーマを強化。

このノートブックでは、具体的なトピックモデリングの手続きや視覚化に重点を置いており、同時にデータの探索的分析（EDA）の結果も考慮しながら、ユーザーの好みに影響を与えうる要素（バイアスや冗長性など）を考察しています。最終的に、特定のトピックがどのように分布し、各トピックがどの文脈において重要であるのかを示すことで、ほしい応答を導く意思決定を支援することを目指しています。

---


# 用語概説 
以下に、Jupyter Notebookの専門用語について、初心者がつまずきそうなマイナーな用語や、このノートブック特有のドメイン知識に焦点を当てて解説します。

1. **BERTopic**:
   - トピックモデリングの手法の一つで、文書をトピックにクラスタリングし、トピックごとに代表的な単語や文書を抽出するためのライブラリ。BERTの埋め込みを利用し、単語と文書の分散表現を組み合わせてトピックを形成します。

2. **HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)**:
   - 密度ベースのクラスタリングアルゴリズムで、データの密度を基にデータポイントをクラスタに分けます。一般的なK-meansクラスタリングのように事前にクラスタ数を定義する必要がなく、異なる密度を持つクラスタを検出できるため、実務での応用において優れています。

3. **c-TF-IDF (class-based Term Frequency-Inverse Document Frequency)**:
   - クラスごとの情報を考慮に入れたTF-IDFの変種。特定のトピックやクラスにおける単語の重要性を評価するための指標で、通常のTF-IDFよりもトピックに関連する情報をより良く反映します。

4. **UMAP (Uniform Manifold Approximation and Projection)**:
   - 高次元データを低次元空間にマッピングする次元削減の手法。データセットのトポロジーを保持しながらデータ点同士の関係性を維持するため、視覚化やクラスタリングの前処理によく使用されます。

5. **Maximal Marginal Relevance (MMR)**:
   - 情報検索の用語で、情報の多様性を最大化しつつ関連性を考慮する手法。ドキュメントの選択や要約に利用され、タスクに基づいて重要な情報を選び出す際に役立ちます。

6. **Bag-of-Words (BoW)**:
   - 文書を単語の集合として表現する方法で、単語の出現頻度をカウントします。文書の順序を無視するため、意味を捉えるのは難しいが、計算が簡単で効率的です。

7. **エンベディング (Embedding)**:
   - 単語や文書をベクトル形式で表現する技術。特に深層学習モデルでよく使用され、語彙の意味的な距離をベクトル空間で表現し、機械学習のモデルに入力可能な形に変換します。

8. **スカイキット (Scikit-learn)**:
   - 機械学習のためのPythonライブラリの一つで、多くの標準的なアルゴリズムやツールが実装されており、データ前処理やモデルの評価、選択などで使用されます。

9. **ランダムシード (Random Seed)**:
   - コンピュータープログラムで生成されるランダムな数値列の初期値。再現性を持たせるために用いることが多く、同じデータセットに対して同じ結果を得るために重要な要素です。

10. **ストップワード (Stop Words)**:
    - テキスト解析において無意味またはあまり意味を持たないとされる一般的な単語（例：あ、か、の、the, isなど）は、分析から除外されることが多いです。これにより、重要な情報が強調され、計算コストが軽減されます。

11. **位置バイアス (Position Bias)**:
    - レコメンデーションシステムや評価において、提示された選択肢の位置によってユーザーの選択が影響を受ける現象。このバイアスを理解することで、より公平な評価が可能になります。

このリストは、初心者がつまずきそうなマイナーな用語や、このノートブック特有の用語に基づいて解説されており、専門的な内容に触れる際の助けとなるでしょう。

---


# インポート（ライブラリの読み込み）

In [ ]:
# IPython.displayモジュールからclear_output関数をインポートします。
# この関数は、Jupyter Notebookの出力をクリアするために使用されます。

In [ ]:
# bertopicというライブラリをインストールします。
# -qオプションは、インストール中の進行状況を表示しないようにするためのものです。
# これにより、インストールのメッセージが少なくなり、ノートブックの出力がスッキリします。

In [ ]:
# numpyライブラリを、バージョン1.23.5にアップグレードしてインストールします。
# -Uオプションは、指定したバージョンがインストールされていない場合や古いバージョンがインストールされている場合に更新することを指示します。
# NumPyは、数値計算や配列操作に非常に便利なライブラリです。

In [ ]:
# pandasライブラリをpdという略称でインポートします。
# literal_eval関数をastモジュールからインポートします。これは、文字列をPythonのリテラルに評価するために使用します。
# numpyライブラリをnpという略称でインポートします。数値計算用のライブラリです。
# osモジュールをインポートします。これは、オペレーティングシステムとのインタラクションに使用します。
# randomモジュールをインポートします。これは、ランダムな数値や選択を生成するために使用します。
# reモジュールをインポートします。これは、正規表現操作を行うために使用します。

# UMAPとHDBSCANは、データの次元削減およびクラスタリングに使用されるライブラリです。
from umap import UMAP
from hdbscan import HDBSCAN

# CountVectorizerとTfidfVectorizerは、テキストデータを数値ベクトルに変換するために使用される方法です。
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# SentenceTransformerは、文章をベクトルに変換するためのモデルを提供するライブラリです。
from sentence_transformers import SentenceTransformer

# BERTopicは、トピックモデリングを行うためのライブラリです。
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

**始める前に、以下はこのノートブックで行われた素晴らしい探索的データ分析（EDA）から得られた概要です：**https://www.kaggle.com/code/abaojiang/lmsys-detailed-eda

**一般的な発見**

- トレーニングセットには64種類の異なるモデルがあります。
- 各プロンプトに対して3つの応答があり、それぞれ異なるモデルからのもので、人間の好みに基づいてランク付けされています。
- トレーニングデータで最も一般的（頻出）なモデルは以下の通りです：

    - gpt-4-1106-preview 
    - gpt-3.5-turbo-0613
    - gpt-4-0613 
    - claude-2.1 
    - (gpt-4-0314, claude-instant-1)
    
    
- ターン数：トレーニングデータのプロンプト/応答ペアの数。

    - 約86.88％の会話はシングルターンです。
    - 99.19％以上の会話は6ターン未満です。
    - 最大ターン数は36です。
    
    

**モデルの応答の好み**

- gpt-3.5-turbo-0314、gpt-4-0125-preview、gpt-4-1106-previewを含む3つのLLMが勝率50％を超えています。
- 低いタイ率は勝者をより決定的に判断できることを意味します。


**応答/プロンプトにおけるバイアスと相関関係**

- 人間の判断者には位置バイアスがありません。つまり、トレーニングデータにおける位置A、B、Cには、アノテータの好みに関する位置バイアスが存在しません。
 
- 同じプロンプトに対するモデルの応答の長さの相関：同じプロンプトに対する応答の長さには強い相関があります。
 
- プロンプトの長さと応答の長さの相関：プロンプトの長さと応答の長さの間には線形関係があるが、相関はかなり弱いようです。

- 冗長性バイアス（応答の冗長さが人間の好みに与える影響）：
    - データには明確な冗長性バイアスがあります。
    - 「平均応答長」と「勝率」の相関 = 0.488
    - gpt-4-0125-previewとgpt-4-1106-previewは、平均応答長が最も長いトップ2のモデルです。
    

**ヌル/空の応答またはプロンプト**

- 空のプロンプトを持つサンプルが5つあります。
- 空のプロンプトはすべて単一のスペース " " で、会話の最後のプロンプトに現れます。
- モデルは空のプロンプトが送信されても応答を続けることができます。
- 欠落した応答は空またはNoneである可能性があります。

- 判断者への影響：
    - タイ率は約0.15に低下し、これは非常に妥当です。
    - もし1つのモデルの応答が欠落している場合、判断者は通常応答する別のモデルに投票したり、タイにする傾向があります。
    

**トレーニングデータとテストデータの確認**

In [ ]:
# 応答オブジェクトを解析するための関数を定義します。
def parse_response(response_object:str)->[str,]:
    # response_objectが文字列として与えられ、それをPythonリテラルに評価しようとします。
    try:
        resp = literal_eval(response_object)
        
    except Exception as e:
        # もし評価に失敗した場合、'null'をNoneに置き換えます。
        try:
            response_object = response_object.replace('null',None)
            resp = literal_eval(response_object)
        except Exception as e:
            # 評価失敗時、デバッグ用に応答オブジェクトの型と内容を出力しますが、コメントアウトされています。
            # print(type(response_object),response_object)
            # 空のリストを返します。
            resp = []

    # 解析された応答を返します。
    return resp

In [ ]:
# トレーニングデータとテストデータを読み込みます。
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
# 'prompt'列の各要素に対してliteral_evalを適用し、文字列をPythonのリテラルに変換します。
train['prompt'] = train['prompt'].apply(literal_eval)
# 'response_a'列と'response_b'列の各要素に対してparse_response関数を適用し、応答を解析します。
train['response_a'] = train['response_a'].apply(parse_response)
train['response_b'] = train['response_b'].apply(parse_response)

# テストデータを読み込みます。
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

# トレーニングデータとテストデータの形状（行数と列数）を出力します。
_ = print(train.shape),print(test.shape)

In [ ]:
# トレーニングデータの最初の5行を表示します。
# これにより、データの構造や内容を確認することができます。
train.head()

In [ ]:
# 空の応答を持つ行を削除します。
# response_aとresponse_bの両方が非空である行のみをフィルタリングします。
train_ss = train[(train.response_a.apply(len)>0) & (train.response_b.apply(len)>0)]

# フィルタリング後のデータの形状（行数と列数）を表示します。
train_ss.shape

In [ ]:
# テストデータの最初の5行を表示します。
# これにより、テストデータの構造や内容を確認することができます。
test.head()

# トピックモデリング

**仕組み (出典: https://maartengr.github.io/BERTopic/algorithm/algorithm.html)**

![image.png](attachment:60ac0df8-5866-4948-9240-7e6453976a10.png)

1. ドキュメントの埋め込み：トランスフォーマーベースのモデル埋め込みを使用して、ドキュメントを数値表現に変換することから始めます。

2. 次元削減：ドキュメントの数値表現を作成した後、これらの表現の次元を削減する必要があります。クラスターモデルは、高次元データを扱う際に「次元の呪い」のために困難を抱えることがよくあります。

3. ドキュメントのクラスタリング：埋め込みを減少させた後、データのクラスタリングを始めることができます。そのために、密度ベースのクラスタリング手法であるHDBSCANを利用します。HDBSCANは、さまざまな形状のクラスターを見つけることができ、可能な場合には外れ値を特定する優れた機能を持っています。

4. Bag-of-words：BERTopicのアルゴリズムでは、モジュラリティを許可しながらトピック表現を作成するためにHDBSCANをクラスタリングモデルとして使用します。これは、異なる密度と形状のクラスターを収容するからです。重心ベースの手法を使用するのではなく、クラスター内のすべてのドキュメントを単一のドキュメントに統合し、各単語の頻度をカウントしてbag-of-words表現を形成します。この表現は、クラスターサイズの違いに対して正規化され、特定のクラスター構造を仮定せずにクラスターレベルでの単語に焦点を当てます。

5. トピック表現：生成されたbag-of-words表現から、1つのクラスターを他のクラスターと区別する要素を知りたいと思います。クラスター1に典型的な単語は何で、他のすべてのクラスターにはあまり典型的でないのはどれでしょうか？これを解決するために、ドキュメントではなくトピック（すなわちクラスター）を考慮するようにTF-IDFを修正する必要があります。

6. トピック表現の微調整：生成されたc-TF-IDFトピックを候補トピックとして考えることができます。それぞれに一連のキーワードと、トピック表現をさらに微調整するために使用できる代表的なドキュメントが含まれます。各トピックに対して代表的なドキュメントのセットを持つことは、大きな利点です。これにより、限られた数のドキュメントの微調整が可能になります。これにより、大規模モデルの計算コストが削減され、各トピックごとにその小さな代表的なドキュメントのセットでのみ操作を行う必要があります。

次に、教師なしトピックモデリングアプローチを使用して、プロンプト内の潜在的なパターンを見つけてみましょう。これにより、プロンプトに含まれるトピックの大まかな理解が得られます。

In [ ]:
# モジュールのセットアップ

# ステップ1 - 埋め込みを抽出する
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# ステップ2 - 次元を削減する
umap_model = UMAP(
    n_neighbors=20,  # 各点の近傍の数
    n_components=5,  # 出力する次元数
    min_dist=0.0,    # 最小距離の制約
    metric="cosine", # コサイン距離を使用
    random_state=7,  # 再現性のためのランダムシード
)

# ステップ3 - 減少した埋め込みをクラスタリングする
hdbscan_model = HDBSCAN(
    min_cluster_size=32,  # 最小クラスタサイズ
    min_samples=1,        # 最小サンプル数
    metric="euclidean",   # ユークリッド距離を使用
    cluster_selection_method="eom", # クラスタ選択メソッド
    prediction_data=True   # クラスタリング結果を予測するためのデータを保持
)

# ステップ4 - トピックをトークナイズする
vectorizer_model = CountVectorizer(stop_words="english") # 英語のストップワードを取り除く

# ステップ5 - トピック表現を作成する
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True,) # 頻出単語を減少させるオプション

# `bertopic.representation`モデルを使用してトピック表現を微調整
representation_model = MaximalMarginalRelevance(diversity=0.4,   # 多様性の調整
                                                top_n_words=15    # トピック表現に含める単語の数
                                               )

In [ ]:
# トピックモデリングのパイプラインを構築します。
topic_model = BERTopic(
    embedding_model=embedding_model,  # 埋め込みモデルを指定
    umap_model=umap_model,            # 次元削減モデルを指定
    hdbscan_model=hdbscan_model,      # クラスタリングモデルを指定
    vectorizer_model=vectorizer_model, # トークナイザーを指定
    ctfidf_model=ctfidf_model,        # c-TF-IDFモデルを指定
    representation_model=representation_model, # 表現モデルを指定
    n_gram_range=(1,5),               # n-gramの範囲を設定（1から5）
    language="english"                # 言語を英語に設定
)

In [ ]:
# トレーニングプロンプトを結合します。
# 各プロンプトの要素を2つの改行で結合し、リストに変換します。
train_prompt_concatenated = train.prompt.apply(lambda x: "\n\n".join(x)).to_list()

# 結合されたプロンプトの数を表示します。
len(train_prompt_concatenated)

In [ ]:
# トピックモデリングの適合を行います。
# train_prompt_concatenatedに基づいてトピックをフィットさせ、トピックとその確率を得ます。
# %%timeマジックコマンドは、このセルの実行時間を計測します。
%%time
topics, topic_proba = topic_model.fit_transform(train_prompt_concatenated)

In [ ]:
# トピックモデルを保存します。
# 保存時に"safetensors"形式を使用します。
topic_model.save("topic_model_unguided", serialization="safetensors")

In [ ]:
# ユニークなトピックの数を表示します。
# np.uniqueを使用して、トピックの配列からユニークな値の数を取得します。
print(f"ユニークなトピックの数: {len(np.unique(topics))}")

In [ ]:
# トピック情報を取得します。
# トピックに関する詳細情報を含むデータフレームを取得します。
topic_info = topic_model.get_topic_info()

# 最初の10行を表示します。
topic_info.head(10)

In [ ]:
# 特定のトピックの表現を取得します。
# トピック0に関する情報を得ます。
topic_model.get_topic(0)

In [ ]:
# トピック表現を視覚化します。
# 上位30のトピックについて、各トピックの上位10単語を表示する棒グラフを作成します。
topic_model.visualize_barchart(top_n_topics=30, n_words=10)

In [ ]:
# トピックの用語ランクを視覚化します。
# トピックにおける単語の重要度のランクを示すプロットを作成します。
topic_model.visualize_term_rank()

In [ ]:
# トピックのヒートマップを視覚化します。
# 上位20トピックの関係性を示すヒートマップを作成します。
topic_model.visualize_heatmap(top_n_topics=20)

# ガイド付きトピックモデリング

以下の図は、ガイド付きトピックモデリングの基本的な考え方を示しています（出典: https://maartengr.github.io/BERTopic/getting_started/guided）。

![image.png](attachment:e350c127-4025-4b0a-9942-3768abb34873.png)

In [ ]:
# タスクトピックを定義します。
# さまざまな自然言語処理タスクに関連するテーマをカンマで区切ってリスト化します。
task_topics = "Code to text,,text to code,Named entity recognition,Sentiment Analysis,Translation,Question Answering,Program Execution, Miscallenous tasks,Text Categorization,Language Identification, Information Extraction,Text Quality,Summarization,text completion,essay writing,poem writing,creative writing,fact verification,reasoning,mathematical,grammer task,rephrasing,style transfer,paraphrasing,natural language inference,question generation,text matching,dialogue generation,harmfullness detection,toxic language detection,fact verification,keyword tagging".split(",")

# 定義したタスクトピックを表示します。
print(task_topics)

**CHATGPTを使用して修正されたタスクキーワード**

In [ ]:
# シードトピックを定義します。
# 修正されたタスクに関連するキーワードのリストを作成します。
task_topics_modified = [
    ['Code to text', 'source code', 'comments', 'explanation', 'description', 'documentation'],
    ['Text to code', 'programming', 'syntax', 'function', 'script', 'automation'],
    ['Named entity recognition', 'NER', 'entities', 'classification', 'annotation', 'identification'],
    ['Sentiment Analysis', 'emotion', 'opinion', 'polarity', 'attitude', 'mood'],
    ['Translation', 'bilingual', 'language pair', 'conversion', 'interpretation', 'localization'],
    ['Question Answering', 'QA', 'response', 'inquiry', 'knowledge', 'retrieval'],
    ['Program Execution', 'run', 'execute', 'compile', 'script', 'process'],
    ['Miscellaneous tasks', 'varied', 'general', 'diverse', 'assorted', 'multiple'],
    ['Text Categorization', 'classification', 'labeling', 'sorting', 'grouping', 'organization'],
    ['Language Identification', 'detection', 'recognition', 'classification', 'language', 'dialect'],
    ['Information Extraction', 'data mining', 'retrieval', 'extraction', 'parsing', 'harvesting'],
    ['Text Quality', 'clarity', 'readability', 'coherence', 'accuracy', 'precision'],
    ['Summarization', 'abstract', 'condense', 'overview', 'digest', 'outline'],
    ['Text completion', 'autocomplete', 'fill-in', 'predictive', 'continuation', 'suggestion'],
    ['Essay writing', 'composition', 'argument', 'thesis', 'structure', 'drafting'],
    ['Poem writing', 'verse', 'rhyme', 'stanza', 'meter', 'lyric'],
    ['Creative writing', 'story', 'imagination', 'narrative', 'fiction', 'expression'],
    ['Fact verification', 'truth', 'validation', 'accuracy', 'confirmation', 'authenticity'],
    ['Reasoning', 'logic', 'deduction', 'inference', 'rationale', 'analysis'],
    ['Mathematical', 'calculation', 'formula', 'equation', 'computation', 'arithmetic'],
    ['Grammar task', 'syntax', 'rules', 'correction', 'structure', 'editing'],
    ['Rephrasing', 'paraphrase', 'reword', 'rewrite', 'restatement', 'alteration'],
    ['Style transfer', 'transformation', 'conversion', 'adaptation', 'modification', 'recasting'],
    ['Paraphrasing', 'rewording', 'restating', 'rephrasing', 'altering', 'modifying'],
    ['Natural language inference', 'NLI', 'hypothesis', 'entailment', 'contradiction', 'inference'],
    ['Question generation', 'inquiry', 'query', 'interrogative', 'ask', 'question'],
    ['Text matching', 'similarity', 'comparison', 'alignment', 'correlation', 'matching'],
    ['Dialogue generation', 'conversation', 'interaction', 'exchange', 'communication', 'chatbot'],
    ['Harmfulness detection', 'toxicity', 'abuse', 'malice', 'danger', 'risk'],
    ['Toxic language detection', 'abusive', 'offensive', 'harmful', 'inappropriate', 'insulting'],
    ['Fact verification', 'validation', 'authenticity', 'accuracy', 'truth', 'confirmation'],
    ['Keyword tagging', 'labeling', 'annotation', 'classification', 'indexing', 'tagging'],
    ['Topic modeling', 'themes', 'topics', 'clustering', 'segmentation', 'grouping'],
    ['Contextual embedding', 'context', 'representation', 'vectors', 'embeddings', 'contextualization'],
    ['Coreference resolution', 'pronouns', 'anaphora', 'antecedents', 'referents', 'binding'],
    ['Semantic similarity', 'meaning', 'relation', 'comparison', 'equivalence', 'likeness'],
    ['Document summarization', 'overview', 'digest', 'abstract', 'compendium', 'condensation'],
    ['Speech recognition', 'transcription', 'audio', 'voice', 'ASR', 'spoken'],
    ['Optical character recognition', 'OCR', 'text', 'image', 'scanning', 'extraction'],
    ['Text generation', 'creation', 'synthesis', 'generation', 'writing', 'production'],
    ['Dialogue summarization', 'conversation', 'overview', 'recap', 'condensation', 'summary'],
    ['Data anonymization', 'privacy', 'masking', 'obfuscation', 'anonymity', 'de-identification']
]

# 修正されたタスクキーワードの数を表示します。
len(task_topics_modified)

In [ ]:
# ガイド付きトピックモデリングのパイプラインを構築します。
# 修正されたタスクキーワードを使ってガイド付きトピックモデルを定義します。
topic_model_guided = BERTopic(
    embedding_model=embedding_model,       # 埋め込みモデルを指定
    umap_model=umap_model,                 # 次元削減モデルを指定
    hdbscan_model=hdbscan_model,           # クラスタリングモデルを指定
    vectorizer_model=vectorizer_model,     # トークナイザーを指定
    ctfidf_model=ctfidf_model,             # c-TF-IDFモデルを指定
    representation_model=representation_model, # 表現モデルを指定
    n_gram_range=(1,5),                    # n-gramの範囲を設定（1から5）
    language="english",                    # 言語を英語に設定
    seed_topic_list=task_topics_modified    # ガイド付きのシードトピックを指定
)

In [ ]:
# ガイド付きトピックモデリングの適合を行います。
# train_prompt_concatenatedに基づいてトピックをフィットさせ、トピックとその確率を得ます。
# %%timeマジックコマンドは、このセルの実行時間を計測します。
%%time
topics, topic_proba = topic_model_guided.fit_transform(train_prompt_concatenated)

In [ ]:
# ガイド付きトピックモデリングのトピック表現を視覚化します。
# 上位30のトピックについて、各トピックの上位10単語を表示する棒グラフを作成します。
topic_model_guided.visualize_barchart(top_n_topics=30, n_words=10)

In [ ]:
# ガイド付きトピックモデルを保存します。
# 保存時に"safetensors"形式を使用します。
topic_model_guided.save("topic_model_guided", serialization="safetensors")

# リソース

* https://research.google/pubs/large-language-models-are-effective-text-rankers-with-pairwise-ranking-prompting/
* https://magazine.sebastianraschka.com/p/tips-for-llm-pretraining-and-evaluating-rms
* https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives
* https://www.kaggle.com/code/abaojiang/lmsys-detailed-eda
* https://www.kaggle.com/code/robikscube/lmsys-chatbot-arena-data-anaylsis#Response-Length-Baseline
* https://medium.com/data-reply-it-datatech/bertopic-topic-modeling-as-you-have-never-seen-it-before-abb48bbab2b2